In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time

THIS_FOLDER = '/Users/nathan/Desktop/Code_Archive/KnowledgeGraph/'
# get webpage

m_to_s = {"march":'spring','april':'spring','may':'spring','june':'summer','july':'summer','august':'summer','september':'fall','october':'fall','november':'fall','december':'winter','january':'winter','february':'winter'}
mn_to_s = {"3":'spring','4':'spring','5':'spring','6':'summer','7':'summer','8':'summer','9':'fall','10':'fall','11':'fall','12':'winter','1':'winter','2':'winter'}

In [ ]:
print('here')

# Federal Reserve Board 

In [6]:
def get_abstracts(url):
    """
        Get abstracts in collapsed sections from input 
        Input: https://www.federalreserve.gov/econres/feds/2021.htm

    """
    dfnew = pd.DataFrame(columns = ['Source','Title','Year', 'Season','Encodings', 'Abstract'])
    # Request the url and parse it with BeautifulSoup
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    good_titles = []
    poss_titles = soup.find_all('a', href=re.compile(r'/econres/feds/'))
    for poss_title in poss_titles:
        title = poss_title['href'].replace('/econres/feds/','').replace('.htm','')
        if len(title)>10 and title.__contains__('.')==False and title.__contains__('//')==False:
            good_titles.append(title)
        
    # Find all the collapsed sections in the document and extract their titles and text content
    collapsed_sections = soup.findAll('div', {'id': re.compile(r'abstract-more\d{5}')})
    date_time = soup.findAll('time')
    
    
    
    for ix, (section, DT) in enumerate(zip(collapsed_sections, date_time)):
        text= section.text.lower()
      
            
        
        #print(text)
        start=text.find('abstract: ')
        end = len(text)
        keywords=[]
        keywords_start=0
        keywords_end = len(text)
        if 'keywords:' in text:
            end = text.find('keywords: ')
            keywords_start = end+len('keywords: ')
        if 'doi:' in text:
            doi = text.find('doi: ')
            if end> text.find('doi: '):
                
                end = doi
            if  keywords_start< doi and keywords != 0:
                keywords_end = doi            
                
        if keywords_start != 0:
            encodings = text[keywords_start:keywords_end].strip('(.|\n)').split(', ')
            
        dt = DT.text.lower().strip().split()
        
        
        abstract = text.lower()[start+len('abstract: '):end]
        
        dfnew = dfnew.append({'Source':'FRB','Title' : good_titles[ix], 'Year' : dt[1], 'Season' : m_to_s[dt[0]], 'Encodings': '', 'Abstract': abstract}, ignore_index = True)
    return dfnew


def extract_links(url):
    """
    Input: url of the page containing links to Policy Briefs
    Output: list of links to Policy Briefs
    """
    base_url = 'https://www.federalreserve.gov'
    dfnew = pd.DataFrame(columns = ['Source','Title','Year', 'Season','Encodings', 'Abstract'])

    # get the html content from the website 
    r = requests.get(url)

    # create a BeautifulSoup object to parse the html content 
    soup = BeautifulSoup(r.content, 'html.parser')

    # find all links on the page and select only those that are relevant for Policy Briefs 
    links = soup.find_all('a', href=re.compile(r'/econres/feds/\d{4}.htm'))
    
    # create a list of links to Policy Briefs 
    link_dict = {}
    #print(links)
    for link in links: 
        if link.contents[0] not in link_dict and link['href'].__contains__('research')==False: 
            #link_dict[link.contents[0]] = link['href']
            
            #print(df)
            dfnew = dfnew.append(get_abstracts(base_url+link['href']))
            
            
            
        break
    return dfnew

df_FRB = extract_links('https://www.federalreserve.gov/econres/feds/all-years.htm')
save = open('FRBAbstracts.txt', mode='w')
save.write('\n'.join(df_FRB['Abstract'].tolist()))

68441

# Peterson Institute 

In [2]:
def get_contents(link):
   
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    date = soup.find('div', class_='field field--date meta meta__date').text
    abstract =  soup.find('div', class_='field field--body').text
    [month, year] = date.lower().split(' ')
    return abstract, month, year
    

def extract_links(url):
    """
    Input: url of the page containing links to Policy Briefs
    Output: list of links to Policy Briefs
    """
    dfnew = pd.DataFrame(columns = ['Source','Title','Year', 'Season','Encodings', 'Abstract'])

    # get the html content from the website 
    r = requests.get(url)

    # create a BeautifulSoup object to parse the html content 
    soup = BeautifulSoup(r.content, 'html.parser')

    # find all links on the page and select only those that are relevant for Policy Briefs 
    links = soup.find_all('a', href=re.compile('/publications/policy-briefs'))

    # create a list of links to Policy Briefs 
    link_dict = {}
    
    for link in links: 
        try:
            if link.contents[0] not in link_dict and link['href'].__contains__('research')==False: 
                #link_dict[link.contents[0]] = link['href']
                #print('https://www.piie.com/'+link['href'])
                abstract, month, year = get_contents('https://www.piie.com'+link['href'])
                dfnew = dfnew.append({'Source':'PIIE','Title' : link.contents[0], 'Year' : year, 'Season' : m_to_s[month], 'Encodings': [], 'Abstract': abstract}, ignore_index = True)
        except:
            print(link)
    return dfnew

PIIE_links = {}
PIIE_reports = pd.DataFrame(columns = ['Title','Year', 'Season','Encodings', 'Abstract'])
page_num = 0
save = open('PIEE.txt', mode='w')
while page_num<100:
    print(page_num)
    PIIE_df = {**PIIE_links,**extract_links('https://www.piie.com/research/publications/policy-briefs?page='+str(page_num))}
        
    #print(PIIE_df['Abstract'][0])
    page_num+=1
    
    save.write('\n'.join(PIIE_df['Abstract'].tolist()))
#PIIE_df.to_csv('JEP_ABSTRACTS.csv', index=True)

    
    
    
#save = open('FRSF.txt', mode='w')
#save.write('\n'.join(PIEE_df['Abstract'].tolist()))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
<a href="/publications/policy-briefs/strategic-importance-us-korea-economic-relations">The Strategic Importance of US-Korea Economic Relations</a>
<a href="/publications/policy-briefs/reforming-overseas-private-investment-corporation-21st-century">Reforming Overseas Private Investment Corporation for the 21st Century</a>
<a href="/publications/policy-briefs/foreign-sales-corporation-reaching-last-act">The Foreign Sales Corporation: Reaching the Last Act?</a>
<a href="/publications/policy-briefs/further-financial-services-liberalization-doha-round">Further Financial Services Liberalization in the Doha Round?</a>
<a href="/publications/policy-briefs/brazil-next">Is Brazil Next?</a>
37
<a href="/publications/policy-briefs/support-ex-im-bank-it-has-work-do">Support the Ex-Im Bank: It Has Work to Do!</a>
<a href="/publications/policy-briefs/sovereign-debt-restructuring-new-articles-new-contr

# Federal Reserve Bank of San Francisco

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time

def get_contents(link):
   
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    date = soup.find('p', {'id':'issue_no'}).text.lower()
    #getting the specific information from the date data. I used a specific strategy
    #just because it worked. General strategy was backup
    month = re.findall(r'[a-z]+',date)[0]
    year  = date[-4:]
    
    poss_texts = soup.findAll('div', class_='m-all t-10of12 d-10of12 t-o-1of12 d-o-1of12')
    text = ''
    for poss in poss_texts:
        text += poss.text

    text = re.sub('[\s\n]{4,}[A-Za-z\s\,\-\.\d]{1,100}[\s\n]{4,}','',text[0:250]) + text[250:]
    #print(link)
    #print(text)
    return text, month, year

def extract_links(url):
    """
    Input: url of the page containing links to Policy Briefs
    Output: list of links to Policy Briefs
    """
    base_url = 'https://www.federalreserve.gov'
    dfnew = pd.DataFrame(columns = ['Source','Title','Year', 'Season','Encodings', 'Abstract'])

    # get the html content from the website 
    r = requests.get(url)
    time.sleep(2)
    # create a BeautifulSoup object to parse the html content 
    soup = BeautifulSoup(r.content, 'html.parser')

    # find all links on the page and select only those that are relevant for Policy Briefs 
    links = soup.find_all('a', href=re.compile(r'https:\/\/www.frbsf.org\/economic-research\/publications\/economic-letter\/\d{4}\/[A-xa-z]+\/[A-xa-z\-]+\/?'))
    
    # create a list of links to Policy Briefs 
    link_dict = {}
    
    for link in links:
        print(link)
        if link['title'] not in link_dict: 
            #print(link['title'])
            link_dict[link['title']] = link['href']
            get_contents(link['href'])
            text, month, year = get_contents(link['href'])
            if year !='2021': 
                print(year)
                dfnew = dfnew.append({'Source':'FRSF', 'Title' : link['title'], 'Year' : year, 'Season' : m_to_s[month], 'Encodings': [], 'Abstract': text}, ignore_index = True)
            #print(link['href'])
            
    return dfnew

dfnew = pd.DataFrame(columns = ['Source','Title','Year', 'Season','Encodings', 'Abstract'])

save = open('FRSF.txt', mode='w')

print(year)
year_url = 'https://www.frbsf.org/economic-research/publications/economic-letter/#'+str(2020)+'/'+str(2020)

dfnew = dfnew.append(extract_links(year_url))

print('\n'.join(dfnew['Abstract'].tolist()))
save.write('\n'.join(dfnew['Abstract'].tolist()))

2000
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2021/november/employment-effects-of-covid-19-across-states-sectors/" itemprop="url" rel="bookmark" title="Employment Effects of COVID-19 across States, Sectors">Employment Effects of COVID-19 across States, Sectors</a>
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2021/november/policymaking-in-time-of-uncertainty/" itemprop="url" rel="bookmark" title="Policymaking in a Time of Uncertainty">Policymaking in a Time of Uncertainty</a>
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2021/november/how-strongly-are-local-economies-tied-to-covid-19/" itemprop="url" rel="bookmark" title="How Strongly Are Local Economies Tied to COVID-19?">How Strongly Are Local Economies Tied to COVID-19?</a>
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2021/november/what-would-it-cost-to-issue-50-year-treasury-bonds/" itemprop="url" re

<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2021/february/can-government-spending-help-to-escape-recessions/" itemprop="url" rel="bookmark" title="Can Government Spending Help to Escape Recessions?">Can Government Spending Help to Escape Recessions?</a>
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2021/january/asymmetric-costs-of-misperceiving-r-star-natural-rate-of-interest/" itemprop="url" rel="bookmark" title="The Asymmetric Costs of Misperceiving R-star">The Asymmetric Costs of Misperceiving R-star</a>
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2020/november/permanent-and-transitory-effects-of-2008-09-recession/" itemprop="url" rel="bookmark" title="Permanent and Transitory Effects of the 2008–09 Recession">Permanent and Transitory Effects of the 2008–09 Recession</a>
2020
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2020/november/small-business-len

2020
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2020/april/news-sentiment-time-of-covid-19/" itemprop="url" rel="bookmark" title="News Sentiment in the Time of COVID-19">News Sentiment in the Time of COVID-19</a>
2020
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2020/march/uncertainty-channel-of-coronavirus/" itemprop="url" rel="bookmark" title="The Uncertainty Channel of the Coronavirus">The Uncertainty Channel of the Coronavirus</a>
2020
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2020/march/why-is-unemployment-currently-so-low/" itemprop="url" rel="bookmark" title="Why Is Unemployment Currently So Low?">Why Is Unemployment Currently So Low?</a>
2020
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2020/february/is-risk-of-lower-bound-reducing-inflation/" itemprop="url" rel="bookmark" title="Is the Risk of the Lower Bound Reducing Inflation?">Is the Risk 

2019
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2019/march/climate-change-and-federal-reserve/" itemprop="url" rel="bookmark" title="Climate Change and the Federal Reserve">Climate Change and the Federal Reserve</a>
2019
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2019/march/modeling-financial-crises/" itemprop="url" rel="bookmark" title="Modeling Financial Crises">Modeling Financial Crises</a>
2019
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2019/february/inflationary-effects-of-trade-disputes-with-china/" itemprop="url" rel="bookmark" title="Inflationary Effects of Trade Disputes with China">Inflationary Effects of Trade Disputes with China</a>
2019
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2019/february/measuring-connectedness-largest-banks/" itemprop="url" rel="bookmark" title="Measuring Connectedness between the Largest Banks">Measuring Connect

2018
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2018/march/do-adjustment-lags-matter-for-inflation-indexed-bonds/" itemprop="url" rel="bookmark" title="Do Adjustment Lags Matter for Inflation-Indexed Bonds?">Do Adjustment Lags Matter for Inflation-Indexed Bonds?</a>
2018
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2018/march/economic-forecasts-with-yield-curve/" itemprop="url" rel="bookmark" title="Economic Forecasts with the Yield Curve">Economic Forecasts with the Yield Curve</a>
2018
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2018/february/monetary-policy-cycles-and-financial-stability/" itemprop="url" rel="bookmark" title="Monetary Policy Cycles and Financial Stability">Monetary Policy Cycles and Financial Stability</a>
2018
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2018/february/do-job-market-networks-help-recovery-from-mass-layoffs/" itemprop

...

2019
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2019/march/climate-change-and-federal-reserve/" itemprop="url" rel="bookmark" title="Climate Change and the Federal Reserve">Climate Change and the Federal Reserve</a>
2019
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2019/march/modeling-financial-crises/" itemprop="url" rel="bookmark" title="Modeling Financial Crises">Modeling Financial Crises</a>
2019
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2019/february/inflationary-effects-of-trade-disputes-with-china/" itemprop="url" rel="bookmark" title="Inflationary Effects of Trade Disputes with China">Inflationary Effects of Trade Disputes with China</a>
2019
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2019/february/measuring-connectedness-largest-banks/" itemprop="url" rel="bookmark" title="Measuring Connectedness between the Largest Banks">Measuring Connect

2018
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2018/march/do-adjustment-lags-matter-for-inflation-indexed-bonds/" itemprop="url" rel="bookmark" title="Do Adjustment Lags Matter for Inflation-Indexed Bonds?">Do Adjustment Lags Matter for Inflation-Indexed Bonds?</a>
2018
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2018/march/economic-forecasts-with-yield-curve/" itemprop="url" rel="bookmark" title="Economic Forecasts with the Yield Curve">Economic Forecasts with the Yield Curve</a>
2018
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2018/february/monetary-policy-cycles-and-financial-stability/" itemprop="url" rel="bookmark" title="Monetary Policy Cycles and Financial Stability">Monetary Policy Cycles and Financial Stability</a>
2018
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2018/february/do-job-market-networks-help-recovery-from-mass-layoffs/" itemprop

2017
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2017/may/reserve-requirements-as-chinese-macro-policy-tool/" itemprop="url" rel="bookmark" title="Reserve Requirements as a Chinese Macro Policy Tool">Reserve Requirements as a Chinese Macro Policy Tool</a>
2017
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2017/may/constructing-home-purchase-sentiment-index/" itemprop="url" rel="bookmark" title="Constructing the Home Purchase Sentiment Index">Constructing the Home Purchase Sentiment Index</a>
2017
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2017/may/preparing-for-next-storm-price-level-targeting-in-low-r-star-world-speech/" itemprop="url" rel="bookmark" title="Preparing for the Next Storm: Reassessing Frameworks and Strategies in a Low R-star World">Preparing for the Next Storm: Reassessing Frameworks and Strategies in a Low R-star World</a>
2017
<a href="https://www.frbsf.org/economic-re

2016
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2016/july/liftoff-and-emerging-markets/" itemprop="url" rel="bookmark" title="Fed Policy Liftoff and Emerging Markets">Fed Policy Liftoff and Emerging Markets</a>
2016
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2016/july/fed-communication-and-zero-lower-bound/" itemprop="url" rel="bookmark" title="Fed Communication and the Zero Lower Bound">Fed Communication and the Zero Lower Bound</a>
2016
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2016/june/do-macroeconomic-variables-help-forecast-interest-rates/" itemprop="url" rel="bookmark" title="Do Macro Variables Help Forecast Interest Rates?">Do Macro Variables Help Forecast Interest Rates?</a>
2016
<a href="https://www.frbsf.org/economic-research/publications/economic-letter/2016/june/energy-prices-and-inflation-expectations/" itemprop="url" rel="bookmark" title="Energy’s Impact on Inflation

KeyboardInterrupt: 

In [13]:
total = ''
combined = open('combined.txt', mode = 'w')
for text in ['BeigeBook.txt', 'PIEE.txt','FRSF.txt', 'FRBAbstracts.txt']:
    lines = open(text, mode='r').readlines()
    for line in lines:
        combined.write(line)

# FRB Beige Book

In [ ]:
from bs4 import BeautifulSoup, NavigableString, Tag


def get_bb_new(url='https://www.federalreserve.gov/monetarypolicy/beigebook202110.htm'):
    base_url = 'https://www.federalreserve.gov'
    
    # get the html content from the website 
    r = requests.get(url)
    
    # create a BeautifulSoup object to parse the html content 
    soup = BeautifulSoup(r.content, 'html.parser')
    divtags = soup.find_all('div', {'id':'article'})
    text = ''
    for tag in divtags:
        tdTags = tag.find_all("p")
        for ix, tag in enumerate(tdTags):
            text += tag.text
            
    return text

def get_bb_old(url='https://www.federalreserve.gov/fomc/beigebook/2007/20070117/FullReport.htm'):
    r = requests.get(url)
    
    soup = BeautifulSoup(r.content, 'html.parser')
    divtags = soup.find_all('p')
    text = ''
    for tag in divtags[5:]:
        if len(tag.text)>100:
            text += tag.text
    return text
    

beige_df = pd.DataFrame(columns = ['Source','Title','Year', 'Season','Encodings', 'Abstract'])
new_beige_url = 'https://www.federalreserve.gov/monetarypolicy/beigebook'
old_beige_url = 'https://www.federalreserve.gov/fomc/beigebook/'
for year in range(2000,2017):
    for month in range(1,13):
        if year <2017:
                for day in range(1,32):
                    daystr=str(day)
                    if day<10:
                        daystr = '0'+str(day)
                    if month >9:
                        datestring = str(year) + str(month) + daystr
                    else:
                        datestring = str(year) + '0' + str(month) + daystr
                    try:
                        #print(datestring)
                        text = get_bb_old(old_beige_url + str(year)+'/' + datestring + '/FullReport.htm')
                        if len(text)>1000:
                            beige_df = beige_df.append({'Source':'FRB', 'Title' : 'Beige Book: ' + datestring, 'Year' : str(year), 'Season' : mn_to_s[str(month)], 'Encodings': ['General'], 'Abstract': text}, ignore_index = True)
                            print(datestring,'-- SUCCESS')

                    except Exception as e:
                        pass
                        
    
    else:       
        for month in range(1,13):
            if year >=2017:
                if month >9:
                    datestring = str(year) + str(month)
                else:
                    datestring = str(year) + '0' + str(month)
                try: 
                    text = get_bb_new(beige_url+datestring+'.htm')
                    if len(text)>1000:

                        beige_df = beige_df.append({'Source':'FRB', 'Title' : 'Beige Book: ' + datestring, 'Year' : str(year), 'Season' : mn_to_s[str(month)], 'Encodings': ['General'], 'Abstract': text}, ignore_index = True)
                        print(datestring,'-- SUCCESS')

                except Exception as e:
                    pass
            
        
     
beige_df.to_csv('JEP_ABSTRACTS.csv', index=True)

In [ ]:
beige_df.to_csv('BeigeBook.csv', index=True)

# Journal of Economic Perspectives

In [ ]:
url = 'https://www.aeaweb.org/journals/jep/issues'

def get_urls_to_issues(url):
    """
    :param url: url to journal issues
    :return: list of urls to journal issues
    """
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    urls = []
    seasons = ['Spring','Summer','Fall','Winter']
    seas = []
    year = []
    for link in soup.find_all('a', href=True):
        
        if re.search('issues/[0-9]+', link['href']):
            urls.append('https://www.aeaweb.org'+link['href'])
            text = link.contents[0].split(' ')
            seas.append(text[0])
            year.append(text[1])
    return urls, seas, year

def get_urls(url):
    """
    extract all urls to articles in issue on webpage:  https://www.aeaweb.org/issues/646
    :param url: str, url to issue
    :return: list of urls to articles in issue
    """

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    # find all links to articles in issue
    links = soup.find_all('a', href=re.compile('^/articles'))
    

    # extract urls from links and add base url
    urls = [link['href'] for link in links]
    titles = [link.contents[0] for link in links]
    
    base_url = 'https://www.aeaweb.org'
    full_urls = [base_url + url for url in urls]
    
    return full_urls, titles

def get_abstract(url):
    
    # get the html from the url
    r = requests.get(url)

    # create a beautiful soup object from the html text
    soup = BeautifulSoup(r.text, 'html.parser')
    codes = extract_jel_codes(soup)
    # find all of the <p> tags in the text and return them as a list of strings (each p tag is one string in the list)
    p_tags = soup.find( class_ = "article-information abstract")
    
    # extract just the text from each of these tags (no tags) and append to a list of strings for each abstract paragraph
    abstract_paragraph = []
    max_len = 0
    if p_tags !=None:
        
        for p in p_tags:  # iterate through all of the paragraphs in this page's abstract section (should be only 1)
            abstract_paragraph.append(p.text)
            
            
        
        if abstract_paragraph != []:
            #print(' '.join(abstract_paragraph), codes)
            
            return ' '.join(abstract_paragraph), codes  # return just the first (and should be only) paragraph as our final result

    return None, codes
    

def extract_jel_codes(soup):

    # find all ul tags with class list-unstyled
    ul_list = soup.find_all('ul', {'class': 'jel-codes'})
    #print(ul_list)
    # find all li tags within each ul tag
    li_list = [ul.find_all('li') for ul in ul_list]
    
    # extract jel code from each li tag and append to list of jel codes
    jel_codes = []
    for lis in li_list:
        
        for li in lis:
            
            for code in re.findall('[A-Z]\d{1,2}',li.text):
                if code not in jel_codes: jel_codes.append(code)
           
    return jel_codes

JEL_df = pd.DataFrame(columns = ['Source','Title','Year', 'Season','Encodings', 'Abstract'])
print('Starting Scrape:\n0/'+str(len(issue_urls)))
for ix,issue in enumerate(issue_urls):
    
    issue_num = int(re.findall('/\d+',issue)[0][1:])
    article_urls,titles = get_urls(issue)
    #print(article_urls)
    for article,title in zip(article_urls,titles):
        ab, codes = get_abstract(article)
        JEL_df = df.append({'Source':'JEP','Title' : title, 'Year' : year[ix], 'Season' : season[ix], 'Encodings': codes, 'Abstract': ab}, ignore_index = True)
    print(str(ix)+'/'+str(len(issue_urls)))
JEL_df.to_csv('JEP_ABSTRACTS.csv', index=True)

# Analysis Section

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from datasets import load_dataset


In [ ]:
train_docs = []
for ix in range(0,len(beige_df)):
    if beige_df['Abstract'][ix] != None:
        doc = beige_df['Abstract'][ix].lower().replace('\t','').replace('\n','').replace(',','').replace('\r','',).replace('.',' ').split(' ')
        train_docs.append(TaggedDocument(doc, [str(beige_df['Title'][ix])]))

In [ ]:
doc_model = Doc2Vec(vector_size=50, window=2, min_count=1, workers=4, epochs=2)

In [ ]:
doc_model.build_vocab(train_docs)

In [ ]:
print(f"Word 'tourism' appeared {doc_model.wv.get_vecattr('bank', 'count')} times in the training corpus.")

"""for ix,doc in enumerate(train_docs):
    print('======\n++++++\n======')
    print('THIS DOC IS TITLED:',doc[1][0],'\n--------')
    
    for near_doc in doc_model.dv.most_similar(doc[1][0]):
        print('Closely related document:', near_doc)
    if ix == 5: break"""

doc_model.wv.most_similar('bank')

In [ ]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer.is_fast

In [ ]:
from transformers import GPT2TokenizerFast
from transformers import AutoTokenizer

dataset = beige_df
dataset = dataset.rename(columns={"Abstract": "text"})
batch_size = 1000
#new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=100000)

In [ ]:
new_tokenizer(dataset["text"][1])


In [ ]:
text = dataset["text"][0]

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')  # or any other checkpoint
word_embeddings = model.transformer.wte.weight  # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings 

#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#text_index = tokenizer.encode('man',add_prefix_space=True)
#vector = model.transformer.wte.weight[text_index,:]

In [ ]:
text_index = tokenizer.encode('interest rate')
vector = model.transformer.wte.weight[text_index,:]
vector

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt


# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
text_index = tokenizer.encode('man',add_prefix_space=True)
vector = model.transformer.wte.weight[text_index,:]

In [ ]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

In [ ]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=100000, special_tokens=special_tokens)
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
sep_token_id = tokenizer.token_to_id("bank")
print(cls_token_id, sep_token_id)

In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

In [ ]:
list(new_tokenizer.vocab.keys())[5000:5020]

In [ ]:
tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [ ]:
import re
import scipy
import pandas as pd
import numpy as np
import nltk
import csv
from nltk.corpus import stopwords
from nltk.corpus import treebank_chunk
from nltk.chunk import ne_chunk
from chunkers import sub_leaves
import matplotlib.pyplot as plt
from nltk.tokenize import PunktSentenceTokenizer
from nltk import pos_tag, ne_chunk
from nltk.tokenize import SpaceTokenizer
import math
import sys
from nltk.corpus import words


csv.field_size_limit(sys.maxsize)
def process_content(tokenized):
    section = []
    
    try:
        print('--')
        for i in tokenized:
            #print(i)
            words = nltk.word_tokenize(i)
            #print('after?')
            tagged = nltk.pos_tag(words)
            chunkGram = r"""Chunk: {<NN>?<NNP>+<NN>?}"""
            chunkParser = nltk.RegexpParser(chunkGram)
            chunked = chunkParser.parse(tagged)
            
            for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Chunk'):
               string = ' '.join([a for (a,b) in subtree.leaves()]).encode('utf-8').strip().decode("utf-8") 
               #print('here', type(string))
               if string.__contains__('GHG') == False and string.__contains__('CDM') == False:
                   section.append(string)
           
    except Exception as e:
        print(str(e))
    
    return section
             

def ProperNounExtractor(text):
    print('PROPER NOUNS EXTRACTED :')
    
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        words = nltk.word_tokenize(text)
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            words = [word for word in words if word not in set(stopwords.words('english'))]
            tagged = nltk.pos_tag(words)
            for (word,tag) in tagged:
                if tag =='NNP':
                    print(word)
                    
def notinenglishdict(text):
    words = set(nltk.corpus.words.words())
    return " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() not in words or not w.isalpha())

def englishornot(text):
    if text not in word.words:
        return True
    else:
        return False



for index,project in enumerate(date_reader):
    #print('num:', project[0][0:-1])
    dates[project[0]] = project[1]
    #if project[0].__contains__('276'):
    #print(project)
    

for rind,row in enumerate(DB[0:1]):
    #if rind>10: break
    if rind ==0:
        new_DB.writerow(row[0:2])
    else:
        newrow = []
        #print(row[0])
        for cind,entry in enumerate(row):
            if cind ==0:
                newrow.append(entry[0:entry.find('-')])
                newrow.append([])
                #print(entry)
            else:
                text = entry
                text = re.sub('[0-9]+',' ',text)
                text = re.sub('[^\w\/\%\’\/]+', ' ', text)
                text = re.sub('/', ' ',text)
                text = re.sub('%', ' ',text)
                text = re.sub('’', ' ',text)
                text = re.sub('WTG', ' ',text)
                text = re.sub('CDM', ' ',text)
                text = re.sub('http', ' ',text)
                text = re.sub('txt', ' ',text)
                text = re.sub('_', ' ',text)
                text = re.sub('’', ' ',text)
                #text = notinenglishdict(text)
                text = re.sub('MW|mW|Mw|mw', ' ',text)
                text = re.sub('WEG|weg|Weg|WeG|wEG|wEg',' ',text)
                     
                 
                tokens = nltk.sent_tokenize(text)
                #print(type(tokens))
                for token in tokens:
                    if token in words.words():
                        tokens.remove(token)
                newsection = process_content(tokens)
                # print('after?')
                newrow[1] += newsection
                
                print(rind, cind)
        #print(type(dates[newrow[0]]))
        newrow.append('\'' +dates[newrow[0]]+'\'')
        new_DB.writerow(newrow)
        

df = pd.read_csv(csv_path)
df_copy = pd.DataFrame(np.chararray(df.shape))
df_copy.index = df.index
df_copy.columns = df.columns
print(df_copy)


In [ ]:
filenames =  ['FRSF.txt', 'cnn.txt','BeigeBook.txt']
with open('combined.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())